In [ ]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

from scipy.stats import itemfreq
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,HashingVectorizer
from sklearn.metrics import confusion_matrix

pd.options.mode.chained_assignment = None

In [ ]:
data1 = pd.read_csv('text_emotion.csv',encoding = "ISO-8859-1")

In [ ]:
data1.head(10)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [ ]:
data1.shape

(40000, 4)

In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   author     40000 non-null  object
 3   content    40000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [ ]:
data1=data1[['tweet_id','sentiment','content']].copy()

In [ ]:
data1.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [ ]:
data1.sentiment = np.where((data1.sentiment == 'neutral') |(data1.sentiment == 'empty')|(data1.sentiment == 'boredom'),'neutral',data1.sentiment)

In [ ]:
data1.sentiment = np.where((data1.sentiment == 'fun') |(data1.sentiment == 'enthusiasm'),'fun',data1.sentiment)

In [ ]:
data1=data1[data1.sentiment !='neutral']

In [ ]:
data1.sentiment.value_counts()

worry        8459
happiness    5209
sadness      5165
love         3842
fun          2535
surprise     2187
relief       1526
hate         1323
anger         110
Name: sentiment, dtype: int64

In [ ]:
data2=pd.read_csv('tweets_clean.txt',sep='	',header=None)

In [ ]:
data2.head(10)

,0,1,2
0,145353048817012736:,Thinks that @melbahughes had a great 50th birt...,:: surprise
1,144279638024257536:,"Como una expresión tan simple, una sola oració...",:: sadness
2,140499585285111809:,the moment when you get another follower and y...,:: joy
3,145207578270507009:,Be the greatest dancer of your life! practice ...,:: joy
4,139502146390470656:,eww.. my moms starting to make her annual rum ...,:: disgust
5,146042696899887106:,If ur heart hurts all the time for tht person ...,:: joy
6,145492569609084928:,"I feel awful, and it's way too freaking early....",:: joy
7,145903955229151232:,So chuffed for safc fans! Bet me dar comes in ...,:: joy
8,142717613234069504:,Making art and viewing art are different at th...,:: fear
9,144183822873927680:,"Soooo dooowwwn!! Move on, get some sleep... Me...",:: anger


In [ ]:
data2.columns=['tweet_id','content','sentiment']

In [ ]:
data2.sentiment = data2.sentiment.str.replace(':: ','')

In [ ]:
data2.sentiment.value_counts()

joy         8240
surprise    3849
sadness     3830
fear        2816
anger       1555
disgust      761
Name: sentiment, dtype: int64

In [ ]:
# Emotions to keep

# worry,happpy(happiness,joy),surprise,sadness,love,fear,anger,hate(disgust+hate),relief,fun(fun+enthusiasm)

In [ ]:
data = data1.append(data2)

In [ ]:
data.head(10)

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,fun,wants to hang out with friends SOON!
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?
11,1956969531,worry,Choked on her retainers
12,1956970047,sadness,Ugh! I have to beat this stupid song to get to...


In [ ]:
data.sentiment = np.where((data.sentiment == 'disgust') |(data.sentiment == 'hate'),'hate',data.sentiment)

In [ ]:
data.sentiment.value_counts()

sadness      8995
worry        8459
joy          8240
surprise     6036
happiness    5209
love         3842
fear         2816
fun          2535
hate         2084
anger        1665
relief       1526
Name: sentiment, dtype: int64

In [ ]:
data=data[data.sentiment.isin(['sadness','worry','joy'])]

In [ ]:
data.sentiment.value_counts()

sadness    8995
worry      8459
joy        8240
Name: sentiment, dtype: int64

In [ ]:
#data3 = pd.read_csv('Equity-Evaluation-Corpus.csv',sep=',')

In [ ]:
#data3.Emotion.value_counts()

## Clean Text

#### Remove irrelevant characters other than alphanumeric and space

In [ ]:
data['content']=data['content'].str.replace('[^A-Za-z0-9\s]+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


#### Remove links from the text

In [ ]:
data['content']=data['content'].str.replace('http\S+|www.\S+', '', case=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


#### Convert everything to lowercase

In [ ]:
data['content']=data['content'].str.lower()

#### Assign Target Variable

In [ ]:
target=data.sentiment
data = data.drop(['sentiment'],axis=1)

In [ ]:
le=LabelEncoder()
target=le.fit_transform(target)

### Split Data into train & test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,target,stratify=target,test_size=0.4, random_state=42)

##### Check if the split divides the classes uniformly

In [ ]:
itemfreq(y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning:     `itemfreq` is deprecated!
    `itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[   0, 4944],
       [   1, 5397],
       [   2, 5075]])

In [ ]:
itemfreq(y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning:     `itemfreq` is deprecated!
    `itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[   0, 3296],
       [   1, 3598],
       [   2, 3384]])

### Tokenization

Tokenization can be done in a variety of ways, namely **Bag of words, tf-idf, Glove, word2vec ,fasttext **etc. Lets see how they can be applied and how they affect the accuracy

#### Bag of Words

In [ ]:
# Extracting features from text files
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.content)
X_test_counts =count_vect.transform(X_test.content)
print('Shape of Term Frequency Matrix: ',X_train_counts.shape)

Shape of Term Frequency Matrix:  (15416, 25747)


#### Naive Bayes Model

In [ ]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
clf = MultinomialNB().fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
nb_clf_accuracy = np.mean(predicted == y_test) * 100
print(nb_clf_accuracy)

59.26250243237984


#### Same thing can be done using a Pipeline

Lets take a look at how it can be done.<br>
First lets define a function for printing accuracy

In [ ]:
def print_acc(model):
    predicted = model.predict(X_test.content)
    accuracy = np.mean(predicted == y_test) * 100
    print(accuracy)

In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

59.26250243237984


#### TF IDF transformer

In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

58.54251799961082


#### Hash Vectorizer

Note: Naive Bayes requires input to be non negative. Therefore, the alternate sign should be set to false in Hashing Vectorizer to make it work with naive bayes algorithm

In [ ]:
nb_clf = Pipeline([('vect', HashingVectorizer(n_features=2500,alternate_sign=False)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

53.76532399299475


In [ ]:
confusion_matrix(y_test,predicted)

array([[2430,  517,  349],
       [ 551, 1989, 1058],
       [ 434, 1278, 1672]])

#### Remove Stop Words

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
58.23117338003503


In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

57.60848414088344


#### Lemmatization

In [ ]:
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])
X_train.loc[:,'content'] = X_train['content'].apply(lemmatize_text)
X_test.loc[:,'content'] = X_test['content'].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

57.60848414088344


In [ ]:
#### Spell Correction with Flashtext

#### Do the same pipeline with NLTK, spacy and pytorch